In [8]:
import time
import matplotlib.pyplot as plt
from generate_data import *
from queue import PriorityQueue
import pandas as pd
import numpy as np
from matplotlib.animation import FuncAnimation
from matplotlib.animation import FuncAnimation, PillowWriter 

start = str(input('start warehouse:'))
end = str(input('end warehouse:'))

list_mw_north, list_mw_central, list_mw_south = create_domain_partition()
[north_mw,central_mw,south_mw],data_neighbour = create_data_IDA()

#phần này tạo data cho visualize
df = pd.read_excel('Data_Excel.xlsx')
lat = {}
long = {}
for i in range(len(df)):
    lat[df.iloc[i][0].strip()] = df.iloc[i][64]
    long[df.iloc[i][0].strip()] = df.iloc[i][65]
    
longtitude = []
latitude = []
province = []
for i in range(len(df)):
    longtitude.append(df.iloc[i][65])
    latitude.append(df.iloc[i][64])
    province.append(df.iloc[i][0])
# Visualize
Longitude, Latitude, Province = df["Longitude"],df["Latitude"],df["Province"]

# we convert matrix distance into list distance
list_distance=[]
for i in df.values:
    list_distance.append(list(i[1:-2]))
# we convert province into index in list distance
dict_index_province={}
for i in range(len(Province)):
    dict_index_province[Province[i].strip()]=i

start_time = time.time()
def IDA(start = start, goal = end):
    list_node_previous={} # solution path
    def IDA_star():
        global list_node_previous
        #define a threshold: theta =f(root_node) with f(n)=h(n)+g(n)
        threshold=list_distance[dict_index_province[start]][dict_index_province[goal]] 
        while True: # run infinity
            temp=search(start,0,threshold,parent='Null') #function search(node,g score,threshold)
            if temp=='FOUND': #if goal found
                return ('FOUND',threshold)
            threshold=temp  
    def search(node,g,threshold,parent):  #recursive function
        f=g+list_distance[dict_index_province[node]][dict_index_province[goal]]
        if (f>threshold):#greater f encountered
            return f
        if (f<=threshold):
            list_node_previous[node]=parent
        if node==goal:   #Goal node found
            return 'FOUND'
        minn=10**10 #minn= Minimum integer
        for tempnode in nextnodes(node):
            #recursive call with next node as current node for depth search
            temp=search(tempnode,g+list_distance[dict_index_province[node]][dict_index_province[tempnode]],threshold,node)
            if temp=='FOUND':# if goal found
                return 'FOUND'
            if (temp<minn):# find the minimum of all f greater than threshold encountered  
                minn=temp
        return minn #//return the minimum f encountered greater than threshold
    def nextnodes(node):
        return data_neighbour[node] #return list of all possible next nodes from node
    print(IDA_star())
    solution_path_optimal=[]
    check=goal
    solution_path_optimal.append(check)
    while check!=start:
        check=list_node_previous[check]
        solution_path_optimal.append(check)
    #print(solution_path_optimal)
    print('Solution path of problem is')
    print(' -> '.join(solution_path_optimal[::-1]))
                                
if start in list_mw_south:
    required_warehouse = south_mw
elif start in list_mw_north:
    required_warehouse = north_mw
elif start in list_mw_central:
    required_warehouse = central_mw
        
def check_same_area_start_warehouse(start,end):   #check if start warehouse and end warehouse are in the same area or not
    if start in list_mw_central and end in list_mw_central:
        return True
    elif start in list_mw_north and end in list_mw_north:
        return True
    elif start in list_mw_south and end in list_mw_south:
        return True
    return False

def distance(provinceA, provinceB):   #return distance between 2 warehouse
    for i in range(len(df)):
        if df.iloc[i,0] == provinceA:
            return df[provinceB].iloc[i]
def find_max_distance(provinceA, provinceB,provinceC):  #find longest edge of triangle
    a = distance(provinceA, provinceB)
    b = distance(provinceB, provinceC)
    c = distance(provinceC, provinceA)
    if a*a >= b*b + c*c:
        return a
    elif b*b >= c*c + a*a:
        return b
    elif c*c >= a*a + b*b:
        return c
    
def rectangular_check(): #with different algorithm, we just need to change algorithm function. E.g: AStar() --> UCS()
    if not check_same_area_start_warehouse(start,end):    #if 2 warehouse are in different area then start warehouse must transfer goods to its area mainwarehouse before shipping to end ware house
        IDA(start,required_warehouse)
        IDA(required_warehouse,end)
    else:
        if distance(start,end) == find_max_distance(start,end,required_warehouse) and start != required_warehouse and end != required_warehouse: 
            IDA(start,required_warehouse)
            DFS(required_warehouse,end)
        else:
            IDA(start,end)

def save_path():  #this function save a gif picture represent scanned warehouse to find optimal path
    %matplotlib inline
    fig = plt.figure()
    fig,axes = plt.subplots(figsize = (10,19))
    for j in range (1,len(latitude)):
        axes.scatter(longtitude[j], latitude[j], c = 'lightblue')
        axes.text(longtitude[j], latitude[j],province[j])
    line, = axes.plot([],[],color = '#33FF00',marker='.',markersize=30)
    def animate(i):
        x = long_check[i]
        y = lat_check[i]
        print(x,y)
        line.set_data(x,y)
        return line,
    anim = FuncAnimation(fig, animate, frames = len(long_check), interval = 300, blit = True)
    anim.save('path.gif', writer = 'pillow')
    
lat_check = []
long_check = []

rectangular_check()

print('time',time.time()-start_time,'s')

start warehouse:Cao Bang
end warehouse:Binh Thuan
('FOUND', 195.7853830416365)
Solution path of problem is
Cao Bang -> Bac Kan -> Thai Nguyen -> Ha Noi
('FOUND', 1278.8103423163816)


KeyboardInterrupt: 